In [0]:
#downlaoding the dataset
!rm transfusion.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data

rm: cannot remove 'transfusion.data': No such file or directory
--2019-03-20 08:25:05--  https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12843 (13K) [text/plain]
Saving to: ‘transfusion.data’

transfusion.data    100%[===================>]  12.54K  --.-KB/s    in 0s      

2019-03-20 08:25:06 (109 MB/s) - ‘transfusion.data’ saved [12843/12843]



In [0]:
import pandas as pd

In [0]:
!ls

sample_data  transfusion.data


In [0]:
import os
os.chdir("/content/")

In [0]:
!ls

sample_data  transfusion.data


In [0]:
#data is read as dataframes
df = pd.read_csv("transfusion.data",delimiter=",")

In [0]:
df.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [0]:
df.tail()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
743,23,2,500,38,0
744,21,2,500,52,0
745,23,3,750,62,0
746,39,1,250,39,0
747,72,1,250,72,0


In [0]:
df.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


In [0]:
#feature separation
feature_columns = ['Recency (months)','Frequency (times)','Monetary (c.c. blood)','Time (months)']
X = df[feature_columns].values
#to be predicted
y = df['whether he/she donated blood in March 2007'].values

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [0]:
#y

In [0]:
#splitting the dataset
from sklearn.model_selection import train_test_split
feature_train,feature_test,label_train,label_test = train_test_split(X,y,test_size=0.2,random_state=15)

In [0]:
#generating a tree
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier()
dtree.fit(X,y)
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [0]:
train_pred = dtree.predict_proba(feature_train)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(14,15))
print(scaler.fit(train_pred))
print(scaler.data_max_)
print(scaler.transform(train_pred))
train_pred_norm = scaler.transform(train_pred)


len(train_pred_norm)

opti_k = []
for i,j in train_pred_norm:
  opti_k.append(max(i,j))

print(opti_k)
#obtained the optimal k value for all the samples in train set by choosing max value

In [0]:
df2 = pd.DataFrame(feature_train,columns =['Recency (months)','Frequency (times)','Monetary (c.c. blood)','Time (months)'])
df2.head(3)
#inserting optimal k as new feature
df2.insert(4,"Optimal K",opti_k)

In [0]:
df2.head()

In [0]:
feature_train = df2.values

In [0]:
feature_train[0]

In [0]:
train_pred = dtree.predict_proba(feature_test)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(14,15))
print(scaler.fit(train_pred))
print(scaler.data_max_)
print(scaler.transform(train_pred))
train_pred_norm = scaler.transform(train_pred)


len(train_pred_norm)

opti_k = []
for i,j in train_pred_norm:
  opti_k.append(max(i,j))

print(opti_k)
#same optimal k is calculated for all samples in test set

df3 = pd.DataFrame(feature_test,columns =['Recency (months)','Frequency (times)','Monetary (c.c. blood)','Time (months)'])
df3.head(3)
df3.insert(4,"Optimal K",opti_k)
#inserted into test features
feature_test = df3.values

print(feature_test)

In [0]:
#knn classifier for predicting whether blood donated in march 2007
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(algorithm="kd_tree",n_jobs=-1,n_neighbors=15,leaf_size=20)
neigh.fit(feature_train,label_train)

In [0]:
import numpy as np
neigh.predict(feature_test)

In [0]:
print("Accuracy is ",neigh.score(feature_test,label_test))

In [0]:
feature_test[1]

In [0]:
label_test[1]

In [0]:
!pip install decision-tree-id3

In [0]:
neigh.get_params()

In [0]:
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
plt.figure(figsize=(10,10))
parallel_coordinates(df, "whether he/she donated blood in March 2007",color=('#ff3300', '#66ff99'))
plt.title('Blood dataset plot', fontsize=20, fontweight='bold')
plt.xlabel('Features', fontsize=15)
plt.ylabel('values', fontsize=15)
plt.legend(loc=1, prop={'size': 15}, frameon=True,shadow=True, facecolor="white", edgecolor="black")
plt.plot(color="red")
plt.show()

In [0]:
from pandas.plotting import andrews_curves
andrews_curves(df, "whether he/she donated blood in March 2007",color=('#ff0000', '#00bbff'))
plt.title('Andrews Curves Plot', fontsize=20, fontweight='bold')
plt.legend(loc=1, prop={'size': 15}, frameon=True,shadow=True, facecolor="white", edgecolor="black")
plt.show()

In [0]:
#feature separation
feature_columns = ['Recency (months)','Frequency (times)','Monetary (c.c. blood)','Time (months)']
X = df[feature_columns].values
#to be predicted
y = df['whether he/she donated blood in March 2007'].values

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [0]:
#splitting the dataset
from sklearn.model_selection import train_test_split
feature_train,feature_test,label_train,label_test = train_test_split(X,y,test_size=0.2,random_state=15)

In [0]:
feature_train

In [0]:
train_pred = dtree.predict_proba(feature_train)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(14,15))
print(scaler.fit(train_pred))
print(scaler.data_max_)
print(scaler.transform(train_pred))
train_pred_norm = scaler.transform(train_pred)


len(train_pred_norm)

th_k = []
for i,j in train_pred_norm:
  th_k.append([i,j])

print(th_k)
#obtained the optimal k value for all the samples in train set by choosing max value

In [0]:
df4 = pd.DataFrame(feature_train,columns =['Recency (months)','Frequency (times)','Monetary (c.c. blood)','Time (months)'])
df4.head(3)
#inserting k1 as new feature
k1 = []
k2 = []
for x in th_k:
  a = x[0] ** (1./2.)
  k1.append(a)
  b = x[1] ** (1./3.)
  k2.append(b)
print("K1 is ",k1)
print("k2 is ",k2)

In [0]:
df4.head(3)

In [0]:
df4.insert(4,"K1",k1)
df4.insert(5,"k2",k2)

In [0]:
df4.head(3)

In [0]:
feature_train = df4.values

In [0]:
test_pred = dtree.predict_proba(feature_test)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(14,15))
print(scaler.fit(test_pred))
print(scaler.data_max_)
print(scaler.transform(test_pred))
test_pred_norm = scaler.transform(test_pred)


len(test_pred_norm)

th_k = []
for i,j in test_pred_norm:
  th_k.append([i,j])

print(th_k)
#same optimal k is calculated for all samples in test set

#inserting k1 as new feature
k1 = []
k2 = []
for x in th_k:
  a = x[0] ** (1./2.)
  k1.append(a)
  b = x[1] ** (1./3.)
  k2.append(b)
#print("K1 is ",k1)


df5 = pd.DataFrame(feature_test,columns =['Recency (months)','Frequency (times)','Monetary (c.c. blood)','Time (months)'])
df5.head(3)
df5.insert(4,"K1",k1)
df5.insert(5,"k2",k2)
#inserted into test features
feature_test = df5.values

print(feature_test)

In [0]:
#knn classifier for predicting whether blood donated in march 2007
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(algorithm="brute",n_neighbors=1,metric="minkowski",weights="distance",p=2)
neigh.fit(feature_train,label_train)

In [0]:
import numpy as np
neigh.predict(feature_test)

In [0]:
print("Accuracy is ",neigh.score(feature_test,label_test))

In [0]:
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
plt.figure(figsize=(10,10))
parallel_coordinates(df, "whether he/she donated blood in March 2007",color=('#ffff00', '#05158c'))
plt.title('Blood dataset plot', fontsize=20, fontweight='bold')
plt.xlabel('Features', fontsize=15)
plt.ylabel('values', fontsize=15)
plt.legend(loc=1, prop={'size': 15}, frameon=True,shadow=True, facecolor="white", edgecolor="black")
plt.plot(color="red")
plt.show()

In [0]:
from pandas.plotting import andrews_curves
andrews_curves(df, "whether he/she donated blood in March 2007",color=('#a500ff', '#ffa100'))
plt.title('Andrews Curves Plot', fontsize=20, fontweight='bold')
plt.legend(loc=1, prop={'size': 15}, frameon=True,shadow=True, facecolor="white", edgecolor="black")
plt.show()